# <font color = 'lightblue'> TRABAJO EDA APRENDIZAJE AUTOMÁTICO


*Guzmán Muñoz Revuelta*

*Fernando Caruana del Pino*

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
# conda install category_encoders
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics

In [15]:
bf_train = pd.read_csv("../data/train_pd_data_preprocessing_missing_outlier.csv")
bf_test = pd.read_csv("../data/test_pd_data_preprocessing_missing_outlier.csv")

In [16]:
bf_train.columns

Index(['Unnamed: 0', 'income', 'name_email_similarity',
       'prev_address_months_count', 'current_address_months_count',
       'customer_age', 'days_since_request', 'intended_balcon_amount',
       'payment_type', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status',
       'credit_risk_score', 'email_is_free', 'housing_status',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month',
       'fraud_bool'],
      dtype='object')

In [17]:
bf_train.dtypes

Unnamed: 0                            int64
income                              float64
name_email_similarity               float64
prev_address_months_count             int64
current_address_months_count          int64
customer_age                          int64
days_since_request                  float64
intended_balcon_amount              float64
payment_type                         object
zip_count_4w                          int64
velocity_6h                         float64
velocity_24h                        float64
velocity_4w                         float64
bank_branch_count_8w                  int64
date_of_birth_distinct_emails_4w      int64
employment_status                    object
credit_risk_score                     int64
email_is_free                         int64
housing_status                       object
phone_home_valid                      int64
phone_mobile_valid                    int64
bank_months_count                     int64
has_other_cards                 

# Codificación de la variable objetivo

In [18]:
bf_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

El código anterior,realiza un conteo de los valores únicos en la columna 'fraud_bool' del DataFrame bf_train. En este caso, muestra el número de ocurrencias para cada valor único en la columna 'fraud_bool'.

El valor 0 aparece 791177 veces.
El valor 1 aparece  8823 veces.

Esto sugiere que la columna 'fraud_bool' es una variable binaria donde 0 podría representar casos no fraudulentos y 1 casos fraudulentos.

In [19]:
X_train = bf_train.drop('fraud_bool',axis=1)
X_test = bf_test.drop('fraud_bool',axis=1)
y_train = bf_train['fraud_bool']
y_test = bf_test['fraud_bool']

train se crea seleccionando todas las columnas de bf_train excepto la columna 'fraud_bool' utilizando drop con axis=1.


X_test se crea de manera similar, seleccionando todas las columnas excepto 'fraud_bool' de bf_test.


y_train se crea asignando la columna 'fraud_bool' de bf_train.


y_test se crea asignando la columna 'fraud_bool' de bf_test.


En resumen, estás separando las características (X) de la variable objetivo (y) tanto para el conjunto de entrenamiento como para el conjunto de prueba. 

# Codificación del resto de variables categoricas

In [20]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

Es una forma de separar las columnas del conjunto de entrenamiento en dos listas: una para las columnas cuyo tipo es "object" o "category", y otra para las columnas restantes.

In [21]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

Este proceso se realiza para convertir las variables categóricas en un formato que sea comprensible para los algoritmos de aprendizaje automático que trabajan con variables numéricas.

In [22]:
model

OneHotEncoder(cols=['payment_type', 'employment_status', 'housing_status',
                    'source', 'device_os'])

In [26]:
X_train_t = model.transform(X_train)
X_test_t = model.transform(X_test)

Aplicamos la transformación de one-hot encoding a los conjuntos de datos de entrenamiento (X_train) y prueba (X_test) utilizando el OneHotEncoder. Al hacerlo tanto X_train como X_test contendrán el conjunto de datos con las columnas categóricas especificadas en list_var_cat.

In [24]:
len(list(X_train_t.columns))

53

Se puede aprecias que el número de columnas ha aumentado despues de aplicar el modelo

In [25]:
X_train_t.dtypes.to_dict()

{'Unnamed: 0': dtype('int64'),
 'income': dtype('float64'),
 'name_email_similarity': dtype('float64'),
 'prev_address_months_count': dtype('int64'),
 'current_address_months_count': dtype('int64'),
 'customer_age': dtype('int64'),
 'days_since_request': dtype('float64'),
 'intended_balcon_amount': dtype('float64'),
 'payment_type_1': dtype('int64'),
 'payment_type_2': dtype('int64'),
 'payment_type_3': dtype('int64'),
 'payment_type_4': dtype('int64'),
 'payment_type_5': dtype('int64'),
 'zip_count_4w': dtype('int64'),
 'velocity_6h': dtype('float64'),
 'velocity_24h': dtype('float64'),
 'velocity_4w': dtype('float64'),
 'bank_branch_count_8w': dtype('int64'),
 'date_of_birth_distinct_emails_4w': dtype('int64'),
 'employment_status_1': dtype('int64'),
 'employment_status_2': dtype('int64'),
 'employment_status_3': dtype('int64'),
 'employment_status_4': dtype('int64'),
 'employment_status_5': dtype('int64'),
 'employment_status_6': dtype('int64'),
 'employment_status_7': dtype('int64'

Después de la codificación one-hot, obtenemos nuevas columnas con nombres como por ejemplo 'payment_type_1', 'payment_type_2', etc., que son del tipo 'int64', indicando variables binarias creadas a partir de una variable categórica original llamada 'payment_type'.